In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import numpy as np

### Loading and Analyzing the Data

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/Mechmay/cardiovascular_disease_prediction_with_Neural_Network/main/heart_failure.csv')

data.head()
print(data.columns)
print(data.shape)
print(data.info())

Index(['Unnamed: 0', 'age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT', 'death_event'],
      dtype='object')
(299, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                299 non-null    int64  
 1   age                       299 non-null    float64
 2   anaemia                   299 non-null    object 
 3   creatinine_phosphokinase  299 non-null    int64  
 4   diabetes                  299 non-null    object 
 5   ejection_fraction         299 non-null    int64  
 6   high_blood_pressure       299 non-null    object 
 7   platelets                 299 non-null    float64
 8   serum_creatinine          299 non-null    float64
 9   ser

In [3]:
print('Classes and number of values in the dataset:', Counter(data['death_event']))
 
y = data.death_event
x = data.iloc[:, 1: -2]
print(x.columns)

Classes and number of values in the dataset: Counter({'no': 203, 'yes': 96})
Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time'],
      dtype='object')


### Data preprocessing

In [4]:
x = pd.get_dummies(x)

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

ct = ColumnTransformer([('numeric', StandardScaler(), ['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time'])])

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)


### Prepare Labels For Classification

In [5]:
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_test = le.transform(Y_test)

# Y_train = to_categorical(Y_train)
# Y_test = to_categorical(Y_test)

### Design the Model

In [6]:
model = Sequential()
model.add(InputLayer(input_shape = (X_train.shape[1],)))
model.add(Dense(12, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

opt = Adam(learning_rate = 0.1)
model.compile(loss= 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

### Train and Evaluate the Model

In [7]:
model.fit(X_train, Y_train, epochs = 300, batch_size = 16, verbose = 1)

loss, acc = model.evaluate(X_test, Y_test, verbose = 0)
print('loss:', loss, 'Accuracy', acc)

Epoch 1/300
15/15 [==============================] - 0s 1ms/step - loss: 0.7599 - accuracy: 0.6052
Epoch 2/300
15/15 [==============================] - 0s 1ms/step - loss: 0.7321 - accuracy: 0.6754
Epoch 3/300
15/15 [==============================] - 0s 2ms/step - loss: 0.6902 - accuracy: 0.6298
Epoch 4/300
15/15 [==============================] - 0s 1ms/step - loss: 0.7435 - accuracy: 0.6345
Epoch 5/300
15/15 [==============================] - 0s 2ms/step - loss: 0.6245 - accuracy: 0.7035
Epoch 6/300
15/15 [==============================] - 0s 2ms/step - loss: 0.6011 - accuracy: 0.7091
Epoch 7/300
15/15 [==============================] - 0s 2ms/step - loss: 0.5996 - accuracy: 0.6810
Epoch 8/300
15/15 [==============================] - 0s 1ms/step - loss: 0.5423 - accuracy: 0.7353
Epoch 9/300
15/15 [==============================] - 0s 1ms/step - loss: 0.4877 - accuracy: 0.7756
Epoch 10/300
15/15 [==============================] - 0s 2ms/step - loss: 0.4667 - accuracy: 0.8057
Epoch 11/

### Generating Classification Report


In [8]:
y_estimate = model.predict(X_test, verbose = 0)
y_estimate = np.argmax(y_estimate, axis = 1)
# Y_true = np.argmax(Y_test, axis = 1)
print(classification_report(Y_test, y_estimate))

              precision    recall  f1-score   support

           0       0.74      0.89      0.81        35
           1       0.78      0.56      0.65        25

    accuracy                           0.75        60
   macro avg       0.76      0.72      0.73        60
weighted avg       0.75      0.75      0.74        60

